In [77]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.utils import to_categorical

In [78]:
X_train_all = []
y_train_all = []

train_files = ['/Users/elhidze/Gagarin/meta_csv/train/anomaly/0.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/anomaly/2.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/anomaly/3.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/anomaly/4.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/anomaly/5.csv',
               
               '/Users/elhidze/Gagarin/meta_csv/train/normal/0.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/1.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/2.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/3.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/4.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/5.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/6.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/7.csv',
               '/Users/elhidze/Gagarin/meta_csv/train/normal/8.csv']
for file in train_files:
    train_df = pd.read_csv(file)
    label_encoder = LabelEncoder()
    train_df['type_encoded'] = label_encoder.fit_transform(train_df['type'])
    X_train_all.append(train_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values)
    y_train_all.append(train_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values)

X_train = np.concatenate(X_train_all)
y_train = np.concatenate(y_train_all)


In [79]:
model = Sequential()
model.add(LSTM(64, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax')) 

/Users/elhidze/Gagarin/venv/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [80]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
test_files = ['/Users/elhidze/Gagarin/meta_csv/test/1.mp4.csv',
              '/Users/elhidze/Gagarin/meta_csv/test/2.mp4.csv',
              '/Users/elhidze/Gagarin/meta_csv/test/3.mp4.csv',
              '/Users/elhidze/Gagarin/meta_csv/test/4.mp4.csv',
              '/Users/elhidze/Gagarin/meta_csv/test/5.mp4.csv'] 
for file in test_files:
    test_df = pd.read_csv(file)
    test_df['type_encoded'] = label_encoder.transform(test_df['type'])
    X_test = test_df[['time_pos', 'type_encoded', 'byte_pos', 'size']].values
    y_test = test_df[['is_normal', 'has_move', 'has_blur', 'has_block', 'has_overexposure']].values
    
    X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

    model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test), verbose=0)

loss, accuracy = model.evaluate(X_test_reshaped, y_test)
print("Test Accuracy:", accuracy)

230/230 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8811 - loss: 1.5230
Test Accuracy: 0.8323336839675903
